In [ ]:
%matplotlib nbagg
import numpy as np
import json
import os, math
import pandas as pd
from matplotlib import pyplot as plt
from scipy.optimize import curve_fit
from IPython.display import clear_output, display

from alvra_tools.load_data import *
from alvra_tools.load_meta import *
from alvra_tools.channels import *
#from alvra_tools.channels_raw import *
from alvra_tools.utils import errfunc_sigma, errfunc_fwhm, threshold
import glob

In [ ]:
scan_name ="monoCalib_2.5kev_10"

DIR_json = "/sf/alvra/data/p18442/res/scan_info/"
DIR = "/sf/alvra/data/p18442/raw/scan_data/" + scan_name + "/"             #use this for a scan

#######################################################################

files_scan = glob.glob(DIR_json + scan_name + '.*')
scan_range = '.'+files_scan[-1].split('.')[-2]
print ('scan_range = {}'.format(scan_range))

In [ ]:
quantile_band = 0.2

###############################################################3

json_file = DIR_json + scan_name + scan_range + ".json"
print (json_file)

with open(json_file) as file:
    data = json.load(file)
numFiles = len(data['scan_files'])
#numFiles = 1

print ("Processing",numFiles,"files")

Signal = []
Izero = []
Energy_eV = []

for i in range(0,numFiles):
    filename = str(data['scan_files'][i][0])
    print (filename)
    exists = os.path.isfile(filename)
    if exists: # and i !=21:
        clear_output(wait=True)
        print("step", i+1, "of",numFiles, f": Processing {filename}")
        (data_trans_shot, data_fluo_shot, Izero_shot, laser_shot, Energy_shot, pulseIDs) = load_FEL_scans(filename, channel_energy)
        
        #df_data = pd.DataFrame(data_fluo_shot)
        df_data = pd.DataFrame(data_trans_shot)
        Signal.append(np.quantile(df_data, [0.5, 0.5 - quantile_band/2, 0.5 + quantile_band/2]))
        
        df_Izero = pd.DataFrame(Izero_shot)
        Izero.append(np.quantile(df_Izero, [0.5, 0.5 - quantile_band/2, 0.5 + quantile_band/2]))
    
        Energy_shot = [x for x in Energy_shot if (np.abs(x) > 0)]
        Energy_eV.append(np.mean(Energy_shot))
    
Signal = np.asarray(Signal)
Izero = np.asarray(Izero)

Energy_eV_readback = load_scan_readback(json_file)
Energy_eV = np.asarray(Energy_eV)
print ("Job done!")
print ('Quantiles for error bars: {},{}'.format(0.5 - quantile_band/2, 0.5 + quantile_band/2))

In [ ]:
# Normalization in case the signal comes from a Diode after I0.

#Signal_norm = -Signal / Izero

# No normalization in case the signal comes from I0 itself.

Signal_norm = Izero

In [ ]:
Int = Signal_norm[:,0]
err_low = Signal_norm[:,1]
err_high = Signal_norm[:,2]

plt.figure()

plt.plot(Energy_eV, Int, label='{}%'.format(quantile_band*100),marker='.')
plt.fill_between(Energy_eV, err_low, err_high, color='lightblue')

plt.title(scan_name + scan_range)
plt.xlabel("Energy (eV)")
plt.legend(loc="best")
plt.grid()

plt.show()

In [ ]:
spectrum_toFit = np.copy(Signal_norm[:,0])
Energy_eV_toFit = np.copy(Energy_eV.ravel())

def gaussian(x, amp, cen, wid):
    return amp * np.exp(-4*(np.log(2))*(x-cen)**2 / wid**2)

# Now fit with error funciton
index = ~(np.isnan(Energy_eV_toFit) | np.isnan(spectrum_toFit))

parameters,extras = curve_fit(gaussian, Energy_eV_toFit[index], spectrum_toFit[index], p0 = [0.3, 2400, 7])
#parameters,extras = curve_fit(errfunc_fwhm, Delay_mm_toFit, Pump_probe_toFit, p0 = [0.01, 0.12, 161, 0.1])

print("Position t0 =",np.round(parameters[1],3),"eV")
print("Width =",np.abs(np.round(parameters[2],3)),"eV")

#best_vals, covar = curve_fit(gaussian, x, y, p0=init_vals)

In [ ]:
plt.figure()

plt.plot(Energy_eV, Int, label='{}%'.format(quantile_band*100),marker='.')
plt.fill_between(Energy_eV, err_low, err_high, color='lightblue')

plt.plot(Energy_eV_toFit, gaussian(Energy_eV_toFit,*parameters))

plt.title(scan_name + scan_range)
plt.grid()
plt.xlabel("Energy (eV)")
plt.legend(loc="best")


plt.show()

In [ ]:
Signal_norm1 = Signal_norm.copy()

In [ ]:
plt.figure()

plt.plot(Energy_eV, Signal_norm_1, label="original",marker='.')
plt.plot(Energy_eV, Signal_norm_3, label="first try",marker='.')
plt.plot(Energy_eV, Signal_norm_2, label="second try",marker='.')

low_err= Signal_norm - err_Signal_norm
high_err= Signal_norm + err_Signal_norm


#plt.fill_between(Energy_eV, low_err, high_err, color='lightblue')
plt.title(DIR)
plt.xlabel("Energy (eV)")
plt.legend(loc="best")


plt.show()
#plt.savefig('Timescan_7_diode')